In [1]:
import json

data = []

with open("/research/d1/gds/jyzhong/computation_models/LLMFramework/data/processed/train_workflow_discovery_abcd_80.json", "r") as f:
    for line in f:
        data.append(json.loads(line))

# Now, let's see the first JSON object
print(data[0].keys())


dict_keys(['sample_id', 'target', 'input', 'input_w_possible_actions', 'input_w_possible_actions_plus', 'target_data'])


In [33]:
with open("/research/d1/gds/jyzhong/computation_models/workflow-discovery/data/raw/guidelines.json", "r") as f:
    guildline = json.load(f)

guildline_flows = []

for key, value in guildline.items():
    for key, flow in value['subflows'].items():
        extracted_flow = ['init']
        for action in flow['actions']:
            action = action['button'].lower().strip().replace(' ', '-')
            extracted_flow.append(action)
        guildline_flows.append(extracted_flow)
        

In [34]:
guildline_flows[0]

['init',
 'pull-up-account',
 'validate-purchase',
 'record-reason',
 'enter-details',
 'offer-refund']

In [35]:
data_mdp = []

for example in data:
    action_graphs = example['target'].split(';')
    action_graphs = [a.split('[')[0].strip().replace(' ', '-') for a in action_graphs]
    action_graphs = ["init"] + action_graphs
    data_mdp.append(action_graphs)

In [36]:
data_mdp[0]

['init',
 'search-faq',
 'search-for-jeans',
 'search-for-jacket',
 'select-topic-in-faq']

In [37]:
from aalpy.learning_algs import run_Alergia

data_mdp = guildline_flows + data_mdp
data_mdp[:10]

model = run_Alergia(data_mdp, 'mc', print_info=True)
model.visualize(path='mdp_guildline_1P.pdf')
model.save('mdp_guildline_1P.dot')

PTA Construction Time:  0.0
Alergia Learning Time: 0.11
Alergia Learned 58 state automaton.
Visualization started in the background thread.
Visualizing 58 state automaton could take some time.


Model saved to mdp_guildline_1P.pdf.


In [51]:
for s in model.states:
    print(s.prefix)
    # print(s.transitions)
    print(s.output)

()
init
('ask-costumer-log-out-log-in',)
ask-costumer-log-out-log-in
('ask-costumer-to-try-again',)
ask-costumer-to-try-again
('check-membership-level',)
check-membership-level
('log-out/in',)
log-out/in
('notify-internal-team',)
notify-internal-team
('notify-team',)
notify-team
('pull-up-account',)
pull-up-account
('pull-up-the-costumer-account',)
pull-up-the-costumer-account
('search-faq',)
search-faq
('try-again',)
try-again
('ask-costumer-log-out-log-in', 'check-instructions')
check-instructions
('ask-costumer-to-try-again', 'enter-details')
enter-details
('log-out/in', 'instructions')
instructions
('pull-up-account', 'ask-the-oracle')
ask-the-oracle
('pull-up-account', 'n/a')
n/a
('pull-up-account', 'record-reason')
record-reason
('pull-up-account', 'shipping-status')
shipping-status
('pull-up-account', 'validate-purchase')
validate-purchase
('pull-up-account', 'verify-identity')
verify-identity
('pull-up-the-costumer-account', 'ask-oracle')
ask-oracle
('pull-up-the-costumer-accou

In [1]:
import re

target = "pull-up-account [david williams];verify-identity [david williams,<zip code>, <zip code>"

def parse_ast_prediction(prediction_str):
    match = re.match(r"(.*)\[(.*)]", prediction_str)
    if match:
        # action w/ value
        action_name = match.group(1).strip()
        slot_str = match.group(2)
        slot_str = slot_str.replace(";", ",")
        slots = [s.strip() for s in slot_str.split(",")]
        '''
        added by xiangyu
        '''
        for i in range(len(slots)):
            if slots[i].endswith(">") and not slots[i].startswith("<"):
                # add "<" to the beginning of the slot
                slots[i] = "<" + slots[i]
            if slots[i].startswith("<") and not slots[i].endswith(">"):
                # add ">" to the end of the slot
                slots[i] = slots[i] + ">"
    else:
        action_name = "MISSING"
        slots = ["MISSING"]

    return action_name, slots

action_name, slots = parse_ast_prediction(target)

In [2]:
print(action_name)
print(slots)

pull-up-account
['david williams']
